In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018', 'Winterthur 2017', 'Winterthur 2018', 'Winterthur 2019', 'Winterthur 2020', 'USZ 2018', 'USZ 2019', 'USZ 2019_30_04_2020', 'KSSG 2021'])

In [3]:
file = FILES_TO_ANALYZE['USZ 2019_30_04_2020']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/USZ_2018-2019_20200730.xlsx', hospital_name_db='USZ', year='2019-2020', sheets=['Rückmeldung_USZ_2019_30.04.2020'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 154 cases for USZ 2019-2020
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [5]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,6130587,10709587,M,54,13,3,E06A,J4400,M200,F058,<NA>,"99b813::20181227,009a13:L:20181231,948x41::201...","99B813::20181227,009A13:L:20181231,948X41::201..."
1,6171196,5863929,M,62,5,3,D62C,J068,M800,"I8028,C770,C787",<NA>,890a11::20190211,890A11::20190211
2,6269659,10160906,W,74,36,3,G72C,R11,M100,R64,<NA>,"948x41::20190607,99c121::20190607","948X41::20190607,99C121::20190607"
3,6329461,10156863,W,79,6,3,B68C,G3531,M100,R471,<NA>,"99c11d::20190708,948x41::20190709","99C11D::20190708,948X41::20190709"
4,6143166,10424299,M,75,5,3,E77E,J690,M100,E43,<NA>,890a21::20190115,890A21::20190115


# match to the database


In [6]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['6130587', '6171196', '6269659', '6329461', '6143166', '6152953',
       '6175818', '6163583', '6157051', '6109664', '6310073', '6149288',
       '6382682', '6421210', '6451720', '6460422', '6444850', '6456868',
       '6461763', '6498608', '6366617', '6450099', '6472113', '6472399',
       '6504693', '6444835', '6485720', '6454834', '6479102', '6442843',
       '6445779', '6505033', '6470578', '6490778', '6463477', '6475938',
       '6477686', '6486979', '6487991', '6501005', '6494118', '6482752',
       '6503090', '6458354', '6472143', '6481583', '6467504', '6504436',
       '6430480', '6470888', '6506865', '6514678', '6516985', '6479305',
       '6485912', '6445940', '6453860', '6456105', '6501187', '6445671',
       '6495041', '6506260', '6506464', '6247973', '6513021', '6473552',
       '6246743', '6129004', '6160852', '6132798', '6159087', '6139918',
       '6238727', '6281216', '6166492', '6253658', '6423485', '6143914',
       '6156358', '6414478', '6438206', '6169777', 

In [7]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl


In [8]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 0 out of 154 revised cases from DtoD that are matched with the database for USZ 2019-2020


In [9]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match